In [ ]:
import pandas as pd
import numpy as np
import pomegranate as pg
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# from sdv.evaluation import evaluate
# from sdv.metrics import KSTest

In [1]:
import pandas as pd
import numpy as np
import pomegranate as pg
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 로드 및 전처리 
pd.set_option('display.max_columns', None)
path = '../../../../../archive/raw/HI-Small_Trans.csv'
df = pd.read_csv(path)

# 2. Timestamp 컬럼을 datetime 객체로 변환 (errors='coerce' 옵션 사용)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M', errors='coerce')

# 3. 'Timestamp' 컬럼에서 NaT인 값 (변환 실패한 값)을 NaN으로 대체
df['Timestamp'] = df['Timestamp'].replace({pd.NaT: np.nan})

# 4. 결측치 처리 (NaN 값을 삭제 또는 대체)
df = df.dropna(subset=['Timestamp'])

# 5. DataFrame이 비어있는지 확인
if df.empty:
    print("DataFrame is empty after removing rows with missing Timestamp values.")
    # 이후 코드를 실행하지 않도록 처리 (예: return)
    exit()

# 6. datetime 객체를 Unix timestamp로 변환
df['Timestamp'] = df['Timestamp'].astype('int64') // 10**9

# (선택 사항) 필요에 따라 스케일링 또는 정규화를 수행
# 예: MinMaxScaler를 사용하여 0과 1 사이의 값으로 스케일링
scaler = MinMaxScaler()
df['Timestamp'] = scaler.fit_transform(df[['Timestamp']])

# 7. Object 타입 컬럼 원핫 인코딩 (대상 컬럼 줄이기)
# 예: 'Receiving Currency', 'Payment Currency', 'Payment Format' 컬럼만 원핫 인코딩
df = pd.get_dummies(df, columns=['Receiving Currency', 'Payment Currency', 'Payment Format'])

# 8. 'Account', 'Account.1' 컬럼은 라벨 인코딩
label_encoder = LabelEncoder()
df['Account'] = label_encoder.fit_transform(df['Account'])
df['Account.1'] = label_encoder.fit_transform(df['Account.1'])

# 9. Bayesian Network 모델 구조 정의 (pomegranate 사용)
# 예시: 간단한 모델 구조
# (실제 데이터에 맞게 모델 구조를 정의해야 합니다.)
model = pg.BayesianNetwork([
    ("From Bank", "Account"),
    ("To Bank", "Account.1"),
    ("Amount Received", "Is Laundering"),
    ("Amount Paid", "Is Laundering"),
    ("Account", "Is Laundering"),
    ("Account.1", "Is Laundering")
])

# 10. 모델 학습
# pomegranate는 데이터 타입을 numpy array 형태로 받음
model = model.fit(df.values)

# 11. 샘플 생성
num_samples = 1000
synthetic_data = []
for i in range(num_samples):
    synthetic_data.append(model.predict([[None] * df.shape[1]])[0])
synthetic_data = np.array(synthetic_data)

# numpy array 를 pandas DataFrame으로 변환
synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)

# 결과 확인
print(synthetic_df.head())

AttributeError: module 'pomegranate' has no attribute 'BayesianNetwork'